In [3]:
import os
import xarray
import s3fs

/Users/aimeebarciauskas/miniconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
def xarray_open_dataset(src_path: str) -> xarray.Dataset:
    """Open dataset."""
    xr_open_args: Dict[str, Any] = {
        "decode_coords": "all",
        "decode_times": False,
        "chunks": None,
        "engine": "h5netcdf"
    }
    fs = s3fs.S3FileSystem()
    file_handler = fs.open(src_path)
    ds = xarray.open_dataset(file_handler, **xr_open_args)
    return ds

In [5]:
%%time
src_path = 's3://nasa-veda-scratch/cmip6-staging/prXin/ACCESS-CM2/prXin-ACCESS-CM2-ssp126.nc'
original_ds = xarray_open_dataset(src_path)

/Users/aimeebarciauskas/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


CPU times: user 988 ms, sys: 409 ms, total: 1.4 s
Wall time: 2.68 s


In [6]:
%%time
src_path = 's3://veda-data-store-staging/rechunking/prXin-ACCESS-CM2-ssp126_compressed.nc'
ds_rechunked_lrg = xarray_open_dataset(src_path)

CPU times: user 1.67 s, sys: 318 ms, total: 1.98 s
Wall time: 15.2 s


In [9]:
%%time
loaded1 = original_ds.load()

CPU times: user 35.5 s, sys: 10.5 s, total: 46 s
Wall time: 4min 31s


In [10]:
%%time
loaded2 = ds_rechunked_lrg.load()

CPU times: user 11.3 s, sys: 1.67 s, total: 12.9 s
Wall time: 16.2 s


In [11]:
display_variable = 'prXin_1'
ds_rechunked = original_ds.chunk(chunks={'time': 1, 'lat': 600, 'lon': 1440})
ds_rechunked[display_variable]

<xarray.DataArray 'prXin_1' (time: 86, lat: 600, lon: 1440)>
dask.array<xarray-prXin_1, shape=(86, 600, 1440), dtype=float64, chunksize=(1, 600, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) int64 0 366 731 1096 1461 ... 29586 29951 30316 30681 31046
Attributes:
    description:  Annual number of days with total precipitation greater than...
    units:        Days

In [13]:
%%time
target_file_name = "prXin-ACCESS-CM2-ssp126_small_chunks_compressed.nc"
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in original_ds.data_vars}
loaded1.to_netcdf(target_file_name, encoding=encoding)

CPU times: user 12.3 s, sys: 63.5 ms, total: 12.3 s
Wall time: 12.4 s


In [14]:
print(f"Compressed netCDF size {os.path.getsize(target_file_name)/1000000} MB")

Compressed netCDF size 23.582471 MB


In [15]:
fs = s3fs.S3FileSystem()
new_target = f"s3://nasa-veda-scratch/cmip6-staging/{target_file_name}"
fs.put(target_file_name, new_target)

[None]

In [16]:
%%time
ds_rechunked_sm = xarray_open_dataset(new_target)

CPU times: user 1.52 s, sys: 298 ms, total: 1.82 s
Wall time: 12.4 s


In [17]:
%%time
ds_rechunked_sm.load()

CPU times: user 12 s, sys: 1.8 s, total: 13.8 s
Wall time: 18 s


<xarray.Dataset>
Dimensions:  (time: 86, lat: 600, lon: 1440)
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) int64 0 366 731 1096 1461 ... 29586 29951 30316 30681 31046
Data variables:
    prXin_1  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_2  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_3  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_4  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    cmip6_source_id:  ACCESS-CM2
    scenario:         ssp126